<a href="https://colab.research.google.com/github/akhiilkasare/Deep-Learning/blob/master/Optimizing_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creating a CNN model and Optimizing with keras tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 5.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=f52bdc4f1709638c4f0fc2a99d951f8d260e1c64aa9a41b91422680d540bfd19
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=e03934ae93fcd5cac1dd247d2c083fe1945039471ada44ae1c41c732575e0468
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
#Scaling down all the images within the range 0 to 1 

train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3324 - accuracy: 0.0994 - val_loss: 2.3045 - val_accuracy: 0.0985
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3040 - accuracy: 0.0997 - val_loss: 2.3048 - val_accuracy: 0.1003
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3039 - accuracy: 0.1003 - val_loss: 2.3050 - val_accuracy: 0.1003


Epoch 1/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.4060 - accuracy: 0.8538 - val_loss: 0.3239 - val_accuracy: 0.8793
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2672 - accuracy: 0.9019 - val_loss: 0.2788 - val_accuracy: 0.8995
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2130 - accuracy: 0.9208 - val_loss: 0.2577 - val_accuracy: 0.9070


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3923 - accuracy: 0.8577 - val_loss: 0.2951 - val_accuracy: 0.8897
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2581 - accuracy: 0.9062 - val_loss: 0.2610 - val_accuracy: 0.9057
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2001 - accuracy: 0.9259 - val_loss: 0.2538 - val_accuracy: 0.9070


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5301 - accuracy: 0.8104 - val_loss: 0.4051 - val_accuracy: 0.8525
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3959 - accuracy: 0.8567 - val_loss: 0.3751 - val_accuracy: 0.8562
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3754 - accuracy: 0.8630 - val_loss: 0.4161 - val_accuracy: 0.8497


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3859 - accuracy: 0.8613 - val_loss: 0.2897 - val_accuracy: 0.8903
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2438 - accuracy: 0.9103 - val_loss: 0.2608 - val_accuracy: 0.9045
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1801 - accuracy: 0.9332 - val_loss: 0.2583 - val_accuracy: 0.9060


INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        13872     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,502,266
Trainable params: 1,502,266
Non-trainable params: 0
_________________________________________________________________


In [15]:
 model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1681 - accuracy: 0.9357 - val_loss: 0.2560 - val_accuracy: 0.9123
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1315 - accuracy: 0.9519 - val_loss: 0.2819 - val_accuracy: 0.9077
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1013 - accuracy: 0.9624 - val_loss: 0.3398 - val_accuracy: 0.9093
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0795 - accuracy: 0.9708 - val_loss: 0.3652 - val_accuracy: 0.9068
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0615 - accuracy: 0.9771 - val_loss: 0.3766 - val_accuracy: 0.9122
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0503 - accuracy: 0.9819 - val_loss: 0.4323 - val_accuracy: 0.9035
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0412 - accuracy: 0.9849 - val_loss: 0.4581 - val_accuracy